NLtoSQL Assistant

In [1]:
!git clone - -recurse-submodules https: // github.com/HKUNLP/UnifiedSKG.git


Cloning into 'UnifiedSKG'...
remote: Enumerating objects: 1171, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 1171 (delta 8), reused 12 (delta 2), pack-reused 1147
Receiving objects: 100% (1171/1171), 21.55 MiB | 10.47 MiB/s, done.
Resolving deltas: 100% (486/486), done.
Submodule 'third_party/sparc' (https://github.com/taoyds/sparc.git) registered for path 'third_party/sparc'
Submodule 'third_party/spider' (https://github.com/elementai/spider) registered for path 'third_party/spider'
Submodule 'third_party/tabert' (https://github.com/facebookresearch/TaBERT.git) registered for path 'third_party/tabert'
Submodule 'third_party/table_pretraining' (https://github.com/microsoft/Table-Pretraining) registered for path 'third_party/table_pretraining'
Submodule 'third_party/test_suite' (https://github.com/elementai/test-suite-sql-eval) registered for path 'third_party/test_suite'
Submodule 'third_party/zero_shot_dst' (https:/

In [2]:
%cd UnifiedSKG


/content/UnifiedSKG


In [3]:
import sys
sys.path.append('/content/drive/My Drive/UnifiedSKG')


In [4]:
!pip install googletrans == 3.1.0a0


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 3.4 MB/s 
     |████████████████████████████████| 42 kB 1.6 MB/s 
     |████████████████████████████████| 1.5 MB 35.0 MB/s 
     |████████████████████████████████| 65 kB 3.4 MB/s 
     |████████████████████████████████| 53 kB 2.3 MB/s 
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16368 sha256=733e32b72c7a7eaf391fc67db2a38da4f847b62f16980f9333f240186b486da8
  Stored in directory: /root/.cache/pip/wheels/dd/59/af/8d6c96a719763990f1c548e36b17d9efdfb767f42f7ff39f53
Successfully built googletrans


In [5]:
!pip install transformers == 4.9.2
!pip install nltk
!pip install datasets == 1.14.0
!pip install sentencepiece
!pip install torch == 1.8.0+cu111 torchvision == 0.9.0+cu111 torchaudio == 0.8.0 - f https: // download.pytorch.org/whl/torch_stable.html


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.6 MB 15.2 MB/s 
     |████████████████████████████████| 3.3 MB 52.6 MB/s 
     |████████████████████████████████| 880 kB 61.5 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=19cd448319441199266ee28cfc352a5d386a8f6770d69a812759deb24e3740d9
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c15899bf659ba74f623ac776e861cf2eb8608c1825ddec66a4
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 290 kB 15.1 MB/s 
     |████████████████████████████████| 212 kB 58.7 MB/s 
     |████████████████████████████████| 56 kB 3.4 MB/s 
     |████████████████████████████████| 132 kB 58.3 MB/s 
  

In [6]:
import os
import time
import torch
import datasets
from googletrans import Translator
from transformers import (
    HfArgumentParser,
    set_seed,
    AutoTokenizer
)
from utils.configue import Configure
from utils.training_arguments import WrappedSeq2SeqTrainingArguments

from filelock import FileLock
import nltk
with FileLock(".lock") as lock:
    nltk.download("punkt", quiet=True)
    nltk.download("stopwords", quiet=True)


In [7]:
sys.argv = ['/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py',

            '--cfg', 'Salesforce/T5_base_prefix_spider_with_cell_value.cfg',
            '--output_dir', './tmp']
parser = HfArgumentParser((WrappedSeq2SeqTrainingArguments,))
training_args, = parser.parse_args_into_dataclasses()
set_seed(training_args.seed)
args = Configure.Get(training_args.cfg)


In [8]:
from models.unified.prefixtuning import Model
tokenizer = AutoTokenizer.from_pretrained(
    "hkunlp/from_all_T5_base_prefix_spider_with_cell_value2", use_fast=False)
model = Model(args)
model.load("hkunlp/from_all_T5_base_prefix_spider_with_cell_value2")


Downloading:   0%|          | 0.00/2.13k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

prefix-tuning sequence length is 10.


Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.01G [00:00<?, ?B/s]

In [11]:
import pickle
filename = 'model.pkl'
pickle.dump(model, open(filename, 'wb'))


In [17]:
def nlq_2_sql():
    translator = Translator()
    inpt = input("Enter a query in natural language:\n")
    question = translator.translate(inpt, dest='en')
    struct_in = "| concert_singer | stadium : stadium_id , location , name , capacity , highest , lowest , average | singer : singer_id , name , country ( France ) , song_name , song_release_year , age , is_male | concert : concert_id , concert_name , theme , stadium_id , year | singer_in_concert : concert_id , singer_id"
    text_in = question.text
    translator = Translator()
    tokenized_txt = tokenizer(
        [text_in], max_length=1024, padding="max_length", truncation=True)
    pred = tokenizer.batch_decode(
        model.generate(
            torch.LongTensor(tokenized_txt.data['input_ids']),
            torch.LongTensor(tokenized_txt.data['attention_mask']),
            num_beams=1,
            max_length=256
        ),
        skip_special_tokens=True
    )
    print("=====SQL Query=====")
    print(*pred)


In [18]:
nlq_2_sql()


Enter a query in natural language:
How many people are over the age of 36?
=====SQL Query=====
select count(*) from people where age > 36


In [20]:
nlq_2_sql()


Enter a query in natural language:
how many people are from india country?
=====SQL Query=====
select count(*) from india


In [21]:
nlq_2_sql()


Enter a query in natural language:
how many singers are lived in france?
=====SQL Query=====
select count(*) from france
